In [1]:
import pandas as pd
import requests
import os
import json

https://python.langchain.com/v0.2/docs/tutorials/rag/

# Data Collection

First, we get the data from the API. As the API is not yet published, both the API-Url and the query to get information on edition-software need to be specified in your .env file. (consult the README for more information)

In [2]:
%load_ext dotenv
%dotenv

In [3]:
# get api_url and query
api_url = os.environ['API_URL']
query = os.environ['QUERY']

# get data from api
api_response = requests.get(api_url + query)

Now that we got the data from the API, we can load it into a dataframe to prepare it to be used as a knowledge base for rag. 

In [4]:
edition_software_info = json.loads(api_response.text)
edition_software_info = pd.DataFrame(edition_software_info)
edition_software_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                25 non-null     object
 1   slug              25 non-null     object
 2   brand_name        25 non-null     object
 3   concept_doi       0 non-null      object
 4   description       22 non-null     object
 5   description_url   3 non-null      object
 6   description_type  25 non-null     object
 7   get_started_url   24 non-null     object
 8   image_id          17 non-null     object
 9   is_published      25 non-null     bool  
 10  short_statement   25 non-null     object
 11  created_at        25 non-null     object
 12  updated_at        25 non-null     object
 13  closed_source     25 non-null     bool  
dtypes: bool(2), object(12)
memory usage: 2.5+ KB


A brief inspection allows us to formulate some initial tasks and questions for this experiment.

- **Preprocessing:** As we can see, not a single entry contains a associated concept_doi. We might consider dropping the column.
- **Impact of using short descriptions only:** Three entries are missing the in depth description. We can assume that rag won't be too useful for these entries. 
- **Impact of additional information:** Only three have a description-url. Down the road, we need to evaluate, if adding info from this source improves the performance of the rag-system.

# Preprocessing

Both the `description` and `short_statement` columns seem to be of particular interest for the task at hand. To asses necessary preprocessing step, we'll need to take a closer look at them.

In [5]:
descriptions = edition_software_info[["description", "short_statement"]]
with pd.option_context('display.max_colwidth', None):
    display(descriptions.head())

,description,short_statement
0,"# Erkennen, Transkribieren und Durchsuchen von historischen Dokumenten mitttels KI\n\n- Trainieren von spezifischen Texterkennungsmodellen, die in der Lage sind, handschriftliche, maschinengeschriebene oder gedruckte Dokumente zu erkennen.\n\n- KI-gestützte Erkennung von handgeschriebenem Text, Layout-Analyse und Strukturerkennung.\n\n- Manuelles Transkribieren im Transkriptionseditor\nKI-gestützten Erkennung mittels öffentlicher oder selbst trainierter KI-Modelle\n\n- Durchsuchen von Dokumenten mit erweiterten Suchoptionen, wie z. B. dem Tool zum Aufspüren von Schlüsselwörtern.\n\n\n- Gemeinsames Arbeiten an Dokumenten, Organisation in Sammlungen\n\n- Teilen von Dokumenten durch eine read&search Website oder Export als PDF oder ALTO (XML).\n\n- Alle Transkribus-Inhalte, d.h. hochgeladene Bilder, erkannte Texte, trainierte Erkennungsmodelle und eingegebene Metadaten, werden innerhalb der EU gehostet und sind GDPR konform.","Transkribus ist eine umfassende Plattform für die Digitalisierung, Texterkennung mithilfe Künstlicher Intelligenz, Transkription und das Durchsuchen von historischen Dokumenten."
1,"autodone is a service for the automated, time-controlled publication of status updates on any Mastodon instance. The codebase is developed under a free license by the Department of Digital Humanities at the University of Cologne and is open to all interested users.\n\nSpecial features of the service include the ability to upload content in tabular format (tsv files) and the ability to publish posts as a thread. In addition to these basic functionalities, more features will be developed in the future.\n\nautodone replaces autoChirp, which offered the same functionality for Twitter before the Twitter API and Twitter itself was massively restricted regarding free and ethical usage.\n\n(quoted from: https://autodone.idh.uni-koeln.de/about, 19.04.2024)\n\n--- \n## Official Site:\n[https://autodone.idh.uni-koeln.de/](https://autodone.idh.uni-koeln.de/)\n\n---\n## Usage Instructions\n[https://autodone.idh.uni-koeln.de/usage](https://autodone.idh.uni-koeln.de/usage)\n","Autodone is a service for the automated, time-controlled publication of status updates on any Mastodon instance. The codebase is developed under a free license by the Department of Digital Humanities at the University of Cologne and is open to all interested users."
2,"[CollateX](http://collatex.net/) is a software to\n\n 1. read **multiple (≥ 2) versions of a text**, splitting each version into parts (tokens) to be compared,\n 1. **identify similarities of and differences between the versions** (including moved/transposed segments) by aligning tokens, and\n 1. output the alignment results in a **variety of formats for further processing**, for instance\n 1. to support **the production of a critical apparatus** or the stemmatical analysis of a text's genesis.\n\nIt resembles software used to compute differences between files (e.g. [diff](http://en.wikipedia.org/wiki/Diff)) or tools for [sequence alignment](http://en.wikipedia.org/wiki/Sequence_alignment) which are commonly used in Bioinformatics. While CollateX shares some of the techniques and algorithms with those tools, it mainly aims for a flexible and configurable approach to the problem of finding similarities and differences in texts, sometimes trading computational soundness or complexity for the user's ability to influence results.\n\nAs such it is primarily designed for use cases in disciplines like [Philology](http://en.wikipedia.org/wiki/Philology) or – more specifically – the field of [Textual Criticism](http://en.wikipedia.org/wiki/Textual_criticism) where the assessment of findings is based on interpretation and therefore can be supported by computational means but is not necessarily computable.\n\nPlease go to <http://collatex.net/> for further information.","CollateX is a software to (a.) read multiple versions of a text, (b.) identify differences by aligning tokens, and (

As we can see, the `description` column contains some formatting artefacts like `\n` and markdown syntax like `**` and `#`. Let's clean them up.
While we're at it, we can also remove double whitespaces etc.

- To do: [CollateX](http://collatex.net/) -> Links
- Markdown vielleicht sogar behalten???

In [6]:
pattern = '\\n+'
edition_software_info["description_clean"] = edition_software_info["description"].str.replace(pattern, '  ', regex=True)

pattern = r'[*#]+|\s-+\s'
edition_software_info["description_clean"] = edition_software_info["description_clean"].str.replace(pattern, '', regex=True)

with pd.option_context('display.max_colwidth', None):
    display(edition_software_info[["brand_name", "description", "description_clean"]].head())

,brand_name,description,description_clean
0,Transkribus,"# Erkennen, Transkribieren und Durchsuchen von historischen Dokumenten mitttels KI\n\n- Trainieren von spezifischen Texterkennungsmodellen, die in der Lage sind, handschriftliche, maschinengeschriebene oder gedruckte Dokumente zu erkennen.\n\n- KI-gestützte Erkennung von handgeschriebenem Text, Layout-Analyse und Strukturerkennung.\n\n- Manuelles Transkribieren im Transkriptionseditor\nKI-gestützten Erkennung mittels öffentlicher oder selbst trainierter KI-Modelle\n\n- Durchsuchen von Dokumenten mit erweiterten Suchoptionen, wie z. B. dem Tool zum Aufspüren von Schlüsselwörtern.\n\n\n- Gemeinsames Arbeiten an Dokumenten, Organisation in Sammlungen\n\n- Teilen von Dokumenten durch eine read&search Website oder Export als PDF oder ALTO (XML).\n\n- Alle Transkribus-Inhalte, d.h. hochgeladene Bilder, erkannte Texte, trainierte Erkennungsmodelle und eingegebene Metadaten, werden innerhalb der EU gehostet und sind GDPR konform.","Erkennen, Transkribieren und Durchsuchen von historischen Dokumenten mitttels KI Trainieren von spezifischen Texterkennungsmodellen, die in der Lage sind, handschriftliche, maschinengeschriebene oder gedruckte Dokumente zu erkennen. KI-gestützte Erkennung von handgeschriebenem Text, Layout-Analyse und Strukturerkennung. Manuelles Transkribieren im Transkriptionseditor KI-gestützten Erkennung mittels öffentlicher oder selbst trainierter KI-Modelle Durchsuchen von Dokumenten mit erweiterten Suchoptionen, wie z. B. dem Tool zum Aufspüren von Schlüsselwörtern. Gemeinsames Arbeiten an Dokumenten, Organisation in Sammlungen Teilen von Dokumenten durch eine read&search Website oder Export als PDF oder ALTO (XML). Alle Transkribus-Inhalte, d.h. hochgeladene Bilder, erkannte Texte, trainierte Erkennungsmodelle und eingegebene Metadaten, werden innerhalb der EU gehostet und sind GDPR konform."
1,Autodone,"autodone is a service for the automated, time-controlled publication of status updates on any Mastodon instance. The codebase is developed under a free license by the Department of Digital Humanities at the University of Cologne and is open to all interested users.\n\nSpecial features of the service include the ability to upload content in tabular format (tsv files) and the ability to publish posts as a thread. In addition to these basic functionalities, more features will be developed in the future.\n\nautodone replaces autoChirp, which offered the same functionality for Twitter before the Twitter API and Twitter itself was massively restricted regarding free and ethical usage.\n\n(quoted from: https://autodone.idh.uni-koeln.de/about, 19.04.2024)\n\n--- \n## Official Site:\n[https://autodone.idh.uni-koeln.de/](https://autodone.idh.uni-koeln.de/)\n\n---\n## Usage Instructions\n[https://autodone.idh.uni-koeln.de/usage](https://autodone.idh.uni-koeln.de/usage)\n","autodone is a service for the automated, time-controlled publication of status updates on any Mastodon instance. The codebase is developed under a free license by the Department of Digital Humanities at the University of Cologne and is open to all interested users. Special features of the service include the ability to upload content in tabular format (tsv files) and the ability to publish posts as a thread. In addition to these basic functionalities, more features will be developed in the future. autodone replaces autoChirp, which offered the same functionality for Twitter before the Twitter API and Twitter itself was massively restricted regarding free and ethical usage. (quoted from: https://autodone.idh.uni-koeln.de/about, 19.04.2024) Official Site: [https://autodone.idh.uni-koeln.de/](https://autodone.idh.uni-koeln.de/) Usage Instructions [https://autodone.idh.uni-koeln.de/usage](https://autodone.idh.uni-koeln.de/usage)"
2,CollateX,"[CollateX](http://collatex.net/) is a software to\n\n 1. read **multiple (≥ 2) versions of a text**, splitting each version into parts (tokens) to be compared,\n 1. **

Next, we isolate links to scrape them later

In [8]:
pattern = r"((?:https?:\/\/|w{3}.)[\w\d%/.-]+)"

edition_software_info["urls"] = edition_software_info["description"].str.extractall(pattern)

with pd.option_context('display.max_colwidth', None):
    display(edition_software_info[["description", "urls"]])


TypeError: incompatible index of inserted column with frame index

In [ ]:
# Add other Info in sentences "Has Licence: XY"